# Steam APIs

In [ ]:
import requests
import json
import os
import time

current_dir = os.getcwd()
operations_dir = os.path.dirname(current_dir)
data_dir = os.path.join(operations_dir, 'Data Management/Landing Zone/Temporal Landing')


def save_json(filename, data, data_dir):
    file_path = os.path.join(data_dir, filename)
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# Create the file if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

urlGetAppList = 'http://api.steampowered.com/ISteamApps/GetAppList/v2'
response = requests.get(urlGetAppList)
allGamesId = response.json()['applist']['apps']

urlAppdetails = "https://store.steampowered.com/api/appdetails"
urlSteamSpy = "https://steamspy.com/api.php?request=appdetails"
urlCurrentPlayers = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1"

appDetails_v1, appSteamSpy_v1, appCurrentPlayers_v1 = {}, {}, {}
appDetails_v2, appSteamSpy_v2, appCurrentPlayers_v2 = {}, {}, {}

# Split the list of games in 2 parts to obtain data of appDetails_v1/v2, appSteamSpy_v1/v2, y appCurrentPlayers_v1/v2
mid_point = len(allGamesId) // 2  

for i in range(len(allGamesId)): 
    appid = str(allGamesId[i]['appid'])
    params1 = {'appids': appid}
    params2 = {'appid': appid}
    print(i, appid)
    successfull = False
    while (not successfull):
        try:
            # Obtain data of each API
            response_app_details = requests.get(urlAppdetails, params=params1)
            response_steamspy = requests.get(urlSteamSpy, params=params2)
            response_current_players = requests.get(urlCurrentPlayers, params=params2)


            # Store the data in the dictionary        
            data_app_details = response_app_details.json()
            if data_app_details[appid]['success']:
                if i < mid_point:
                    appDetails_v1[appid] = data_app_details[appid]
                else:
                    appDetails_v2[appid] = data_app_details[appid]
                    

            data_steamspy = response_steamspy.json()
            if data_steamspy['name'] != '':
                if i < mid_point:
                    appSteamSpy_v1[appid] = data_steamspy
                else:
                    appSteamSpy_v2[appid] = data_steamspy


            data_current_players = response_current_players.json()
            if data_current_players["response"]["result"] == 1:
                if i < mid_point:
                    appCurrentPlayers_v1[appid] = data_current_players
                else:
                    appCurrentPlayers_v2[appid] = data_current_players
                    
        except Exception as e:
            print(f"Error al procesar el juego con appid {appid}: {e}")
            successfull = False
            time.sleep(150)  # Retry after a delay in case of failure
        else:
            print(f"Procesando el juego con appid {appid} correctamente.")
            successfull = True

    if i%1000 == 0:
        save_json('steam_api_details_data_v1.json', appDetails_v1, data_dir)
        save_json('steam_api_details_data_v2.json', appDetails_v2, data_dir)

        save_json('steam_api_steamspy_data_v1.json', appSteamSpy_v1, data_dir)
        save_json('steam_api_steamspy_data_v2.json', appSteamSpy_v2, data_dir)

        save_json('steam_api_current_players_data_v1.json', appCurrentPlayers_v1, data_dir)
        save_json('steam_api_current_players_data_v2.json', appCurrentPlayers_v2, data_dir)

    

# Store the data in 2 different versions
save_json('steam_api_details_data_v1.json', appDetails_v1, data_dir)
save_json('steam_api_details_data_v2.json', appDetails_v2, data_dir)

save_json('steam_api_steamspy_data_v1.json', appSteamSpy_v1, data_dir)
save_json('steam_api_steamspy_data_v2.json', appSteamSpy_v2, data_dir)

save_json('steam_api_current_players_data_v1.json', appCurrentPlayers_v1, data_dir)
save_json('steam_api_current_players_data_v2.json', appCurrentPlayers_v2, data_dir)

print(mid_point)
print(len(allGamesId))

print(len(appDetails_v1))
print(len(appDetails_v2))

print(len(appSteamSpy_v1))
print(len(appSteamSpy_v2))

print(len(appCurrentPlayers_v1))
print(len(appCurrentPlayers_v2))
